<i><b>Public AI</b></i>
<br>
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 2. Semantic Segmentation**
# Section 2. Simple Segmentation 


### _Objective_
1. Semantic Segmentation 을 CNN 을 이용해 구현하는 가장 기본적인 원리를 이해하고 구현합니다. 
2. **Fashion MNIST Segmentation 만들기** : Keras를 활용하여 간단한 Simple Semantic Segmentation 모델을 구성해 보도록 합니다.


In [ ]:
# If You use in Colab, You Should run this script
import os
if (not os.path.exists("./multicampus_segmentation_generator") and
    not "multicampus_segmentation_generator" in os.getcwd()):
    !git clone https://github.com/public-ai/multicampus_segmentation_generator.git
    os.chdir("./multicampus_segmentation_generator")

In [ ]:
from tensorflow.keras.layers import Layer, Conv2D, MaxPool2D, Conv2DTranspose, ZeroPadding2D
from tensorflow.keras.layers import Input, UpSampling2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.layers import Softmax, Add
from tensorflow.keras.layers import Lambda, concatenate
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import optimizers
import tensorflow.keras.backend as K 
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt 
from keras.utils import np_utils
import numpy as np
import tensorflow as tf
import cv2
import sys 
sys.path.append('../')
from generator import fasion_mnist_generator

#  1. Segmentation 을 위한 Fashion MNIST Dataset 생성하기 

28 x 28 Pixel 로 이루어진 Fashion Mnist Image 이용해 Segmentation 데이터 셋을 생성합니다. <br>


데이터셋 입력 데이터의 shape 는 128 x 128 로 기본 설정 되어 있으며 <br>
또한 기존의 흑백 이미지 객체에 색을 입혀 3차원 이미지로 변경했습니다.  
입력 데이터셋의 Shape 는 (128, 128, 3) 입니다.

결과(y)의 Shape 는 (128, 128, 11) 입니다.
11은 기존의 클래스 갯수(10) 에 배경 정보 까지 더한 결과 입니다. 



![Imgur](https://i.imgur.com/x3FIAtC.png)

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fasion_mnist_generator(1000, 10)
print('Train Image shape : ', train_images.shape)
print('Train Labels shape : ', train_labels.shape)
print('Test Image shape : ', test_images.shape)
print('Test Labels shape : ', test_labels.shape)

## [ Show Sample Images ] 

생성된 Segmentation Mnist Image 5개의 정답 이미지를 시각화 해봅니다. <br>


In [ ]:
for img_ind in range(5):
    fig, axes = plt.subplots(1,12)
    fig.set_size_inches((30,10))
    sample_image = train_images[img_ind]
    axes[0].imshow(train_images[img_ind])
    axes[0].set_title('Image {}'.format(img_ind))
    axes[0].axis('off')

    for i in range(1,12):
        sample_mask = train_labels[img_ind][..., i-1]
        axes[i].imshow(sample_mask)
        axes[i].set_title('{}'.format(i))
        axes[i].axis('off')

    plt.show()

# 2. Semantic Segmentation 이해하기

![Imgur](https://i.imgur.com/H3jkXlN.png)

- **Classification** : Image 을 넣어서 해당 이미지가 어떤 숫자인지를 분류 해야 합니다..  

- **Semantic Segmentation** : Image 을 넣어서 각 이미지 픽셀이 어떤 Class 인지 결정 해야 합니다..<br> 
마지막 logits 출력층의 크기는 class 갯수 + 배경 class 입니다. 즉 이번 예제에서는 10 + 1 임으로 
출력층의 Shape 는 (None, **11**) 입니다. 


# 3. Simple Segmentation 구현하기
![Imgur](https://i.imgur.com/AoM6XiM.png)




In [ ]:
from tensorflow.python.keras.layers import Conv2D, Conv2DTranspose, MaxPool2D, Input, Add
from tensorflow.python.keras.models import Model
tf.reset_default_graph()

# Inputs
# fix me! # input과 class 갯수 n_classes를 설정해주세요

# Build Models
# Block 1, Image shape (128, 128) -> (128, 128)
# fix me! # 위 표에 만족하는 Convolution Neural Network를 구성해주세요

# Block 2 Output Shape : (128, 128) -> (128, 128)
# fix me! # 위 표에 만족하는 Convolution Neural Network를 구성해주세요

# Block 3 Output Shape :(128, 128) -> (128, 128)
# fix me! # 위 표에 만족하는 Convolution Neural Network를 구성해주세요

# Get Logits : 마지막 출력에는 각 pixel 별로 배경을 포함해 각 클래스를 Classification 합니다
# fix me! # 마지막 출력 layer를 구성하고 model을 생성해주세요

## Metrics

In [ ]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

## Train

In [ ]:
# optimizer, loss function, metric을 지정하는 compile 과정을 진행해주세요!
# optimizer 로는 Nesterov Accelerated Gradient(NAG)를 사용해주세요
# learning_rate=0.01, decay=1e-6, momentum=0.9
# fix me!

# batch_size는 1, epochs=1로 설정하여 학습을 진행해주세요!
# fix me!

## Test

In [ ]:
test_loss, test_mean_iou = model.evaluate(test_images/255., test_labels)
test_preds = model.predict(test_images/255.)
test_cls = np.argmax(test_preds, axis=-1)

## Show Test Result 

In [ ]:
for img_id in range(len(test_cls)):
    fig, axes = plt.subplots(1, 11)
    fig.set_size_inches((30,10))
    axes = np.array(axes).ravel()
    for i in range(11):
        axes[i].axis('off')
        axes[i].imshow(test_cls[img_id] == i)
    
    plt.show()

---
⊙ Copyright(c) 2020 by PublicAI. All rights reserved <br>
All pictures, codes, writings cannot be copied without permission. <br>
Writen by PAI(info@publicai.co.kr) <br>
last updated on 2020/01/4 <br>

---